# Exponential Smoothing On Dataset of Energy sector

In [4]:
import pickle
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
import numpy as np
import itertools

Loading dataset

In [5]:
df = pd.read_csv("energy.csv")
print(df)

                   Datetime  PJME_MW
0       2002-12-31 01:00:00  26498.0
1       2002-12-31 02:00:00  25147.0
2       2002-12-31 03:00:00  24574.0
3       2002-12-31 04:00:00  24393.0
4       2002-12-31 05:00:00  24860.0
...                     ...      ...
145361  2018-01-01 20:00:00  44284.0
145362  2018-01-01 21:00:00  43751.0
145363  2018-01-01 22:00:00  42402.0
145364  2018-01-01 23:00:00  40164.0
145365  2018-01-02 00:00:00  38608.0

[145366 rows x 2 columns]


Checking for missing values

In exponential smoothing standardization and normalization is not required

In [6]:
missing_values = df.isnull().sum()

print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Datetime    0
PJME_MW     0
dtype: int64


Applying model

In [7]:
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)

def find_best_ets_model(data, seasonal_periods):
    trend_options = ['add', 'mul']
    seasonal_options = ['add', 'mul']
    
    best_params = None
    best_aic = np.inf

    for trend, seasonal in itertools.product(trend_options, seasonal_options):
        for period in seasonal_periods:
            model = ExponentialSmoothing(data, trend=trend, seasonal=seasonal, seasonal_periods=period).fit()
            aic = model.aic
            
            if aic < best_aic:
                best_aic = aic
                best_params = (trend, seasonal, period)

    best_model = ExponentialSmoothing(data, trend=best_params[0], seasonal=best_params[1], seasonal_periods=best_params[2]).fit()
    return best_model, best_params, best_aic

best_model, best_params, best_aic = find_best_ets_model(df['PJME_MW'], seasonal_periods=[24])

print("Best Parameters (Trend, Seasonal, Seasonal Periods):", best_params)
print("Best AIC:", best_aic)

C:\Users\Esha\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Esha\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Esha\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Esha\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Esha\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

Best Parameters (Trend, Seasonal, Seasonal Periods): ('add', 'mul', 24)
Best AIC: 1995306.8432935502


Plotting

In [8]:
# Plot original data and predictions
predictions = best_model.predict(start=df.index[0], end=df.index[-1])
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['PJME_MW'], label='Original Data', color='purple')
plt.plot(df.index, predictions, label='Predictions', color='pink')
plt.title('Exponential Smoothing (ETS) Model with Error, Trend, and Seasonal Components (Optimized)')
plt.xlabel('Date')
plt.ylabel('PJME_MW')
plt.legend()
plt.grid(True)
plt.show()

In [9]:
import pickle
with open('est_energy.pkl', 'wb') as f:
    pickle.dump(find_best_ets_model, f)